In [ ]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# Define tropical diseases and common diseases in Africa
diseases = [
    "Anthrax", "Avian Influenza", "Chikungunya", "Cholera", "COVID-19", "Crimean-Congo Haemorrhagic Fever", "Dengue", "Ebola",
    "Hepatitis B Virus", "Hepatitis C", "Hepatitis E Virus", "HIV", "Lassa Fever", "Malaria", "Marburg Virus Disease", "Measles",
    "Meningococcal Meningitis", "MERS (Middle East Respiratory Syndrome)", "Monkeypox (Mpox)", "Plague", "Polio (Poliomyelitis)",
    "Rift Valley Fever", "Tuberculosis", "Yellow Fever", "Zika Virus"
]

In [ ]:
# Define the correct local path
file_path = r"C:\Users\User\Desktop\Project\CDC.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Display the first few rows
df.head()

In [ ]:
# Assuming `ds` is your existing DataFrame
ds = pd.DataFrame(columns=["URL", "Description"])

# Set up Chrome WebDriver
driver = webdriver.Chrome()

# Step 1: Open the Africa CDC diseases page
base_url = "https://africacdc.org/disease/"
driver.get(base_url)
time.sleep(5)  # Allow page to load

# Step 2: Collect all disease URLs
disease_links = []
disease_elements = driver.find_elements(By.XPATH, '//*[@id="wpv-view-layout-3438-TCPID3223"]/div[*]/div/div/div[2]/a')
for element in disease_elements:
    disease_links.append(element.get_attribute("href"))

# Step 3: Scrape data from each disease page and add to `ds`
for link in disease_links:
    driver.get(link)  # Open the disease page
    time.sleep(3)  # Allow page to load

    try:
        # Extract the information using XPath
        description_element = driver.find_element(By.XPATH, '/html/body/div[2]/section[2]/div/div/div/section/div/div/div/div/div/p[3]')
        description = description_element.text
    except Exception as e:
        description = "No information found"  # Handle cases where the XPath does not exist

    # Add data to the DataFrame `ds`
    new_row = pd.DataFrame({"URL": [link], "Description": [description]})
    ds = pd.concat([ds, new_row], ignore_index=True)

# Save the updated DataFrame `ds` to a CSV
output_file = "diseases_info.csv"
ds.to_csv(output_file, index=False)

# Print completion message
print(f"Scraping completed! Data saved to {output_file}")

# Close the browser
driver.quit()

In [ ]:
ds

In [ ]:
new_df = pd.concat([df.iloc[:, 0], ds.iloc[:, :2]], axis=1)
new_df

In [ ]:
new_df = pd.concat([df.iloc[:, 0], ds.iloc[:, :2]], axis=1)
new_df = new_df.rename(columns={"Description": "Symptoms"})
new_df

In [ ]:
new_df.to_csv("symptoms_done.csv", index=False)